In [ ]:
pip install gmaps

In [ ]:
import pandas as pd ## importing librarys
import matplotlib.pyplot as plt
import folium
import requests
import geopandas as gpd
from shapely.geometry import Point, Polygon
import geopy.distance
import random
import numpy as np
from sklearn.utils import shuffle
import gmaps
from datetime import datetime
import collections
collections.Iterable = collections.abc.Iterable
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:

class TripInformation(object):

      """
      This class represents trip information, including details like city, state, days, latitude, and longitude.

      Attributes:
        city (str): The name of the city for the trip.
        state (str): The state or region where the trip is taking place.
        lat (float): The latitude coordinate of the trip location.
        lng (float): The longitude coordinate of the trip location.
        days (int): The number of days planned for the trip.
        restaurant_type (int): Type of restaurants the user wants.

      Methods:
        execute(): Execute all the functions for trip planning in a defined order.
        city_input(): Prompt the user for the city they want to visit and retrieve information about the city.
        days_input(): Prompt the user for the number of days of the trip.
        restaurant_type_input(): Prompt the user for the type of restaurant they would like to visit.
        get_coordinates(city): Uses Google API to get name, state, latitude, longitude of the city prompted by the user.

     """

      def __init__(self):
          self.city = None
          self.lat = None
          self.lng = None
          self.days = None
          self.restaurant_type = None


      def execute(self):
          self.city_input()
          self.days_input()
          self.restaurant_type_input()


      def city_input(self):

          city  = input("What city would you like to visit? ")
          city, state, lat, lng = self.get_coordinates(city)

          self.city = city
          self.state = state
          self.lat = lat
          self.lng = lng



      def days_input(self):

          while True: # Prompts the user until a valid number of days is provided
              user_input = input(f"For how many days will you visit {self.city}? ")
              if user_input.isdigit(): # Check if the input is a valid integer
                  self.days = int(user_input)
                  break
              else:
                  print("Please, type a number")

      def restaurant_type_input(self):

          while True: # Prompts the user until a valid restaurant type is provided
              user_input = input("What type of restaurant would you like to go?\n(1) Average Restaurants\n(2) Fast-Food\n(3) Michelin Restaurants\n(4) All of the above\n")
              if user_input.isdigit(): # Check if the input is a valid integer
                  self.restaurant_type = int(user_input)
                  break
              else:
                  print("Please, type a number")


      @staticmethod
      def get_coordinates(city_name):

          api_key = 'API_KEY' # Google API to get the city informationn

          base_url = 'https://maps.googleapis.com/maps/api/geocode/json'

          params = {
              'address': city_name,
              'key': api_key,
          }

          response = requests.get(base_url, params=params) # Make the request
          data = response.json()

          city = ''

          if data['status'] == 'OK': # Verify if the data status is valid
              location = data['results'][0]['geometry']['location']
              lat = location['lat']
              lng = location['lng']
              city = data["results"][0]["address_components"][0]["long_name"]
              state = data["results"][0]["address_components"][2]["long_name"]
              return city,state, lat, lng # Returns city, state, latitude and longitude
          else: # If not, return None
              print(f"Geocoding request failed with status: {data['status']}")
              return None


In [ ]:
class Hotels(object):

      """
      This class represents a collection of hotels and provides methods to filter and rank them based on criteria.

      Attributes:
        trip (TripInformation): An instance of the TripInformation class that contains trip details.
        df (DataFrame): A DataFrame to store Hotel data.

      Methods:
        execute(): Execute a series of tasks to filter and rank hotels.
        read_csv(): Read hotel data from a CSV file and preprocess it.
        calculate_distance(): Calculate the distance between hotels and the trip's location.
        top_hotels(): Determine the top-rated hotels within a certain distance from the trip's location.
      """

      def __init__(self, trip):
          self.trip = trip
          self.df = pd.DataFrame()


      def execute(self):
          self.read_csv()
          self.calulate_distance()
          self.top_hotels()

      def read_csv(self):
          self.df = pd.read_csv("Datafiniti_Hotel_Reviews_Jun19.csv")
          self.df.drop(["id", "dateAdded", "dateUpdated", "address", "categories", "primaryCategories", "keys",
                              "websites", "reviews.userCity", "reviews.userProvince", "reviews.username", "sourceURLs",
                              "reviews.title", "reviews.text", "reviews.sourceURLs", "reviews.dateSeen",
                              "reviews.dateAdded", "reviews.date"], axis=1, inplace=True)

      def calulate_distance(self):
          # uses geopy to calculate the distance of each of the hotels to the city of the trip
          self.df['distance'] = self.df.apply(lambda row: geopy.distance.geodesic((self.trip.lat, self.trip.lng), (row['latitude'], row['longitude'])).km, axis=1)


      def top_hotels(self):

          hotels = self.df

          city_hotels = hotels[hotels["distance"] < 15] # Filter the hotels to distance less than 15km
                                                        # To only get hotels located in the city

          hotel_ids = []
          n_reviews = city_hotels['name'].value_counts()  # Gets the number of reviews of each hotel
          id_hotels = city_hotels['name'].value_counts().index # Gets the id of each of this hotels

          new_df = city_hotels.groupby('name').mean().reset_index()
          new_df["Number of reviews"] = 0

          for i in range(len(id_hotels)):
              ind = new_df[new_df['name'] == id_hotels[i]]["Number of reviews"].index[0]
              new_df.at[ind, "Number of reviews"] = n_reviews[i]

          # Calculates the Hotel score (0.2 * Number of reviews * Hotel Rating * 0.8)
          new_df["Hotel Score"] = new_df["reviews.rating"] * .8 + new_df["Number of reviews"] * .2

          # Sorts the Hotels by Score
          self.df = new_df.sort_values("Hotel Score", ascending=False)

          # Filter hotels by selecting those with a score at least 80% of the highest-rated hotel
          self.df = self.df[self.df["Hotel Score"] > self.df.iloc[0]["Hotel Score"] * 0.8]

          # Chooses one random hotel from the top, randomizing some of the recommendation
          self.df = self.df.iloc[random.randint(0, self.df.shape[0]-1)]


In [ ]:
class Locations:

    """
    This class represents a collection of locations and provides methods to filter and rank them based on criteria.

    Attributes:
        trip (TripInformation): An instance of the TripInformation class that contains trip details.
        df (DataFrame): A DataFrame to store Location data.

    Methods:
        execute(): Execute a series of tasks to filter and rank locations.
        read_csv(): Read location data from a CSV file and preprocess it.
        calculate_distance(): Calculate the distance between locations and the trip's location.
        top_locations(): Determine the top-rated locations based on criteria such as proximity and check-ins.
    """

    def __init__(self, trip):
        self.trip = trip
        self.df = pd.DataFrame()

    def execute(self):
        self.read_csv()
        self.calculate_distance()
        self.top_locations()

    def read_csv(self):
        self.df = pd.read_csv("Location.csv")
        self.df.rename(columns={"Latitude": "latitude", "Longitude":"longitude", "Location":"name"}, inplace=True)

    def calculate_distance(self):
        self.df['distance'] = self.df.apply(lambda row: geopy.distance.geodesic(
            (self.trip.lat, self.trip.lng), (row['latitude'], row['longitude'])).km, axis=1)

    def top_locations(self):

        location = self.df

        # Eliminating city and state names from location names for improved recommendation accuracy
        location['name'] = location['name'].str.replace(f"{self.trip.city} City", '')
        location['name'] = location['name'].str.replace(f"{self.trip.city},", '')
        location['name'] = location['name'].str.replace(f"{self.trip.city}", '')
        location['name'] = location['name'].str.replace(f"{self.trip.state},", '')
        location['name'] = location['name'].str.replace(f"{self.trip.state}", '')

        # Filter locations based on proximity and name length criteria
        filtrado = location[(location["distance"] < 20) & (location['name'].str.len() > 5)]

        # Calculate the count of check-ins for each location
        filtrado['Count'] = filtrado.groupby('name')['name'].transform('count')

        # Drop duplicates
        top = filtrado.drop_duplicates(subset='name')

        # Sets the Location DF and sorts it by the number of "check-ins"
        self.df = top.sort_values("Count", ascending=False)

        # Gets the top N * 4 locations and then chosses N random ones, to randomize some of the recommendation
        self.df = self.df.iloc[0 :self.trip.days*4]
        self.df = self.df.sample(n=(self.trip.days*2))




In [ ]:
class Restaurant:

    """
    This class encapsulates all the other restaurant related classes.

    Attributes:
        trip (TripInformation): An instance of the TripInformation class that contains trip details.
        fastfood (FastFood): An instance of the FastFood class for fast-food restaurant-related tasks.
        michelin (Michelin): An instance of the Michelin class for Michelin-starred restaurant-related tasks.
        normal (Normal): An instance of the Normal class for regular restaurant-related tasks.
        restaurant_qtd (list): A list to store the quantity of each restaurant type for the recommendation.

    Methods:
        execute(): Execute tasks related to fast-food, Michelin-starred, and regular restaurants.
        calculate_distances(): Calculate the distance between restaurants and the trip's location.
        restaurant_types(): Decide the quantity of each restaurant type for the recommendation.

    """

    def __init__(self, trip):
        self.trip = trip
        self.fastfood = None
        self.michelin = None
        self.normal = None
        self.restaurant_qtd = []


    def execute(self):
        self.fastfood = FastFood(self.trip)
        self.fastfood.execute()

        self.michelin = Michelin(self.trip)
        self.michelin.execute()

        self.normal = Normal(self.trip)
        self.normal.execute()

        self.restaurant_types()

        self.normal.restaurant_qtd = self.restaurant_qtd
        self.michelin.restaurant_qtd = self.restaurant_qtd

        if(self.restaurant_qtd[0] > 0):
            self.normal.favorite_culinarys_input()
            self.michelin.df = self.michelin.df[self.michelin.df['cuisine'].apply(lambda x: any(x in types for types in self.normal.df['Why']))]

        if(self.trip.restaurant_type == 3):
            self.michelin.user_input()




    def calculate_distances(self):
        self.df['distance'] = self.df.apply(lambda row: geopy.distance.geodesic(
            (self.trip.lat, self.trip.lng), (row['latitude'], row['longitude'])).km, axis=1)

    def restaurant_types(self):
        # function to decide the quantity of each restaurant type for the recommendation

        # If restaurant_type == 1 (Only normal Restaurants)
        if self.trip.restaurant_type == 1:
            self.restaurant_qtd = [self.trip.days, 0, 0]

        # If restaurant_type == 2 (Only FastFood)
        elif self.trip.restaurant_type == 2:
            self.restaurant_qtd = [0, self.trip.days, 0]

        # If restaurant_type == 3 (Only Michelin)
        elif self.trip.restaurant_type == 3:
            # Extra "if" inside the Michelin, as not all cities have michelin restaurants
            # If the city does not have one, it prints the message and replaces it for normal restaurants.
            if self.trip.days > self.michelin.df.shape[0]:
                print(f"Not enough Michelin Restaurants in {self.trip.city}, replacing for normal restaurants.")
                self.restaurant_qtd = [self.trip.days- self.michelin.df.shape[0],
                                       self.michelin.df.shape[0],
                                       0]
            else:
                self.restaurant_qtd = [0,0, self.trip.days]
        # If restaurant_type == 4 (All 3 types)
        elif self.trip.restaurant_type == 4: # Checks if there is Michelin Restaurants in the city
            if(round(0.2 * self.trip.days) <= self.michelin.df.shape[0]):
                self.restaurant_qtd = [round(0.6 * self.trip.days), round(0.2 * self.trip.days), round(0.2 * self.trip.days)]
            else: # If there is Michelin Restaurants, the division of the amount is this:
                # Normal restaurants:   60% of days
                # FastFood restaurants: 20% of days
                # Michelin restaurants: 20% of days
                self.restaurant_qtd = [round(0.6 * self.trip.days) + round(0.2 * self.trip.days) -self.michelin.df.shape[0]
                    , self.michelin.df.shape[0], round(0.2 * self.trip.days)]


class FastFood(Restaurant):

    """
    This class represents a collection of fast-food restaurants and inherits functionality from the Restaurant class.

    Attributes:
        trip (TripInformation): An instance of the TripInformation class that contains trip details.
        df (DataFrame): A DataFrame to store fast-food restaurant data.

    Methods:
        execute(): Execute tasks related to fast-food restaurants, including reading data and filtering top fast-food restaurants.
        read_csv(): Read fast-food restaurant data from a CSV file and preprocess it.
        top_fastfood(): Determine the top-rated fast-food restaurants based on proximity to the trip's location.

    """

    def __init__(self, trip):
        super().__init__(trip)
        self.df = pd.DataFrame()

    def execute(self):
        self.read_csv()
        self.top_fastfood()

    def read_csv(self):
        self.df = pd.read_csv("Datafiniti_Fast_Food_Restaurants.csv")
        self.df.drop(["id", "dateAdded", "dateUpdated", "keys", "sourceURLs", "websites"], axis=1, inplace=True)
        self.df["type"] = "Fast Food"

    def top_fastfood(self):
        super().calculate_distances() # Calculates the distance between the restaurants and the city
        self.df.sort_values("distance", inplace=True) # Sorts the values based on the distace
        self.df = self.df[self.df["distance"] < 15] # Filters the df for only restaurants in a 15km radius of the city
        self.df = self.df[0: self.trip.days *4].sample(self.trip.days * 2)

class Michelin(Restaurant):

    """
    This class represents a collection of Michelin-starred restaurants and inherits functionality from the Restaurant class.

    Attributes:
        trip (TripInformation): An instance of the TripInformation class that contains trip details.
        df (DataFrame): A DataFrame to store Michelin-starred restaurant data.

    Methods:
        execute(): Execute tasks related to Michelin-starred restaurants, including reading data and filtering top Michelin restaurants.
        read_csv(): Read Michelin-starred restaurant data from CSV files, preprocess it, and filter by regions in the United States.
        top_michelin(): Determine the top-rated Michelin-starred restaurants based on proximity to the trip's location.
    """

    def __init__(self, trip):
        super().__init__(trip)
        self.df = pd.DataFrame()

    def execute(self):
        self.read_csv()
        self.top_michelin()

    def read_csv(self):

        one_star = pd.read_csv("one-star-michelin-restaurants.csv")
        two_star = pd.read_csv("two-stars-michelin-restaurants.csv")
        three_star = pd.read_csv("three-stars-michelin-restaurants.csv")

        one_star["star"] = 1
        two_star["star"] = 2
        three_star["star"] = 3

        self.df = pd.concat([one_star, two_star, three_star])

        self.df = self.df[(self.df["region"] == "California") | (self.df["region"] == "New York City") |
                          (self.df["region"] == "Chicago") | (self.df["region"] == "Washington DC")]

        self.df["type"] = "Michelin"

    def top_michelin(self):
        super().calculate_distances() # Calculates the distance between the restaurants and the city
        self.df.sort_values("distance", inplace=True) # Sorts the values based on the distace
        self.df = self.df[self.df["distance"] < 15] # Filters the df for only restaurants in a 15km radius of the city

    def user_input(self):
        top_cuisines = self.df['cuisine'].value_counts().head((self.trip.days * 2)).index.tolist()


        print("Do you like this cuisines? ")
        for i, cuisine in enumerate(top_cuisines):
            print(f"({i+1}) {cuisine}")
        favorites = input(f"Which are your favorites? (Choose {self.restaurant_qtd[2]}, space separated) ")

        favorites = favorites.split()
        favorites = [int(i) for i in favorites]

        # Turns the number from the user input into the cuisines names
        cuisines_list = [top_cuisines[i-1] for i in favorites]

        preferred_cuisines_df = pd.DataFrame({'cuisine': cuisines_list})
        filtered_df = self.df[self.df['cuisine'].isin(cuisines_list)]


        new_df = pd.DataFrame()
        restaurant_names = []
        self.df.reset_index(inplace=True)

        for cuisine in cuisines_list: # iterate over the cuisines the user likesz

            cuisine_df = self.df[ # Gets all restaurants that offer this cuisine
                self.df["cuisine"].str.contains(cuisine)]

            for index in cuisine_df.index: # Iterates over this cuisine df
                if self.df._get_value(index, "name") not in restaurant_names:
                    # If the restaurant is not alredy in the recommended list
                    # Put it on the recommended list and go to the next cuisine
                    restaurant_names.append(cuisine_df._get_value(index,"name"))
                    row = pd.DataFrame([cuisine_df.loc[index]])
                    row["Why"] = cuisine
                    new_df = pd.concat([new_df,row], ignore_index=True)
                    break

        self.df = new_df

class Normal(Restaurant):

    """
    This class represents a collection of normal restaurants and inherits functionality from the Restaurant class.

    Attributes:
        trip (TripInformation): An instance of the TripInformation class that contains trip details.
        df (DataFrame): A DataFrame to store normal restaurant data.
        restaurant_qtd (list): A list to store the quantity of each restaurant type for the recommendation.

    Methods:
        execute(): Execute tasks related to normal restaurants, including reading data, geocoding zip codes,
                  and filtering top-rated normal restaurants.
        read_csv(): Read normal restaurant data from a CSV file, preprocess it, and add additional columns.
        get_lat_lng(): Geocode zip codes to retrieve latitude and longitude coordinates for restaurants.
        top_normal_restaurants(): Determine the top-rated normal restaurants based on ratings, reviews, and proximity.
        favorite_culinarys_input(): Get user preferences for restaurant cuisine.

    Static Methods:
        famous_cuisines(df): Calculate famous culinary types from a DataFrame.
    """

    def __init__(self, trip):
        super().__init__(trip)
        self.df = pd.DataFrame()
        self.restaurant_qtd = []


    def execute(self):
        self.read_csv()
        self.get_lat_lng()
        self.top_normal_restaurants()


    def read_csv(self):
        self.df = pd.read_csv("TripAdvisor_RestauarantRecommendation.csv")

        # Making new individualized columns for City, State and Zip_code
        self.df['State'] = self.df['Location'].str.split(',').str[1].str[1:3]
        self.df['City'] = self.df['Location'].str.split(',').str[0].str.strip()
        self.df["Zip"] =  self.df['Location'].str.split(',').str[1].str[3:] #
        self.df["zip_code"] = self.df["Zip"].str[:6]


        self.df['zip_code'] = pd.to_numeric(self.df['zip_code'], errors='coerce') # Transforms from object to int
        self.df.dropna(subset=['zip_code'], inplace=True) # If not possible to transform, drop.
        self.df["zip_code"] = self.df["zip_code"].astype(int)

        # Drops unnecessary columns
        self.df.drop(["Comments", "Contact Number", "Trip_advisor Url", "Menu", ],axis=1,inplace=True)

        # Determinates the type of restaurant "Normal"
        self.df["type"] = "Normal"

    def get_lat_lng(self):

        """
          The Normal Restaurants do not have Latitude and Longitude, only ZIP codes
          This function transforms this ZIP codes into latitude and logitude using a csv.
        """

        zips_csv = pd.read_excel("uszips.xlsx")
        zips_csv = zips_csv.rename(columns={"zip":"zip_code"})

        self.df.set_index("zip_code", inplace=True)
        zips_csv.set_index("zip_code", inplace=True)

        self.df = self.df.merge(zips_csv[['lat', 'lon']], left_on="zip_code",right_on="zip_code", how='left')
        self.df.rename(columns={"lat": "latitude", "lon":"longitude", "Name":"name"}, inplace=True)


    def top_normal_restaurants(self):

        # Makes a colum for the restaurant reviews
        self.df['Reviews'] = [r.split()[0] for r in self.df.Reviews]#
        self.df['Reviews'] = pd.to_numeric(self.df['Reviews'], errors='coerce')
        self.df.dropna(subset=['Reviews'], inplace=True)

        # Makes a column for the restaurant number of Reviews
        self.df['No of Reviews'] = [n.split()[0].replace(',', '') for n in self.df['No of Reviews']]
        self.df['No of Reviews'] = pd.to_numeric(self.df['No of Reviews'], errors='coerce')
        self.df.dropna(subset=['No of Reviews'], inplace=True)
        self.df["No of Reviews"] = self.df["No of Reviews"].astype(int)

        self.df.dropna(inplace=True)
        self.df.drop_duplicates(keep="first", inplace=True)


        super().calculate_distances() # Calculate the distances

        # Calcultas the restaurant score based on the reviews and distance to the city
        self.df["Restaurant Score"] = self.df["Reviews"] * 0.9 + self.df["No of Reviews"] * 0.1 / self.df["distance"]

        # Sort the DataFrame by the Restaurant Score
        self.df.sort_values("Restaurant Score", ascending=False,inplace=True)

        # Filters the DataFrame for only restaurants in a 15km radius of the city
        self.df = self.df[self.df["distance"] < 15]

        # Gets the top restaurants, as they are alredy sorted, only gets the top self.trip.days * x
        self.df = self.df[0: self.trip.days *5]


    @staticmethod
    def famous_cuisines(df):
        # Splits the restaurant cuisines
        df['Types'] = df['Type'].str.split(',').apply(lambda x: [s.strip() for s in x] if isinstance(x, list) else [])

        all_cuisine = [culinary_type for sublist in df['Types'] for culinary_type in sublist]

        # Make a dictonary of each cuisine and the number od restaurants that offer them
        cuisine_type_counts = {}
        for culinary_type in all_cuisine:
            if culinary_type in cuisine_type_counts:
                cuisine_type_counts[culinary_type] += 1
            else:
                cuisine_type_counts[culinary_type] = 1

        # Makes a DataFrame out of the dict and sorts it
        cuisine_df = pd.DataFrame(list(cuisine_type_counts.items()), columns=['Culinary Type', 'Count'])
        cuisine_df = cuisine_df.sort_values(by='Count', ascending=False)

        return cuisine_df


    def favorite_culinarys_input(self):

        # Gets the top cuisines (Number of restaurants that offer them)
        famous_cuisines = self.famous_cuisines(self.df)

        # Gets the top N * 2
        cuisines = famous_cuisines["Culinary Type"].to_list()[:self.restaurant_qtd[0]*2]

        # Gets the user input of his favorite cuisines.
        print("Do you like this cuisines? ")
        for i, cuisine in enumerate(cuisines):
            print(f"({i+1}) {cuisine}")

        favorites = input(f"Which are your favorites? (Choose {self.restaurant_qtd[0]}, space separated) ")
        favorites = favorites.split()
        favorites = [int(i) for i in favorites]

        # Turns the number from the user input into the cuisines names
        cuisines_list = [cuisines[i-1] for i in favorites]

        new_df = pd.DataFrame()
        restaurant_names = []
        self.df.reset_index(inplace=True)

        for cuisine in cuisines_list: # iterate over the cuisines the user likesz

            cuisine_df = self.df[ # Gets all restaurants that offer this cuisine
                self.df["Type"].str.contains(cuisine)]

            for index in cuisine_df.index: # Iterates over this cuisine df
                if self.df._get_value(index, "name") not in restaurant_names:
                    # If the restaurant is not alredy in the recommended list
                    # Put it on the recommended list and go to the next cuisine
                    restaurant_names.append(cuisine_df._get_value(index,"name"))
                    row = pd.DataFrame([cuisine_df.loc[index]])
                    row["Why"] = cuisine
                    new_df = pd.concat([new_df,row], ignore_index=True)
                    break

        # The normal restaurants df gets this new df.
        self.df = new_df


In [ ]:
class Recomendation(object):

    """
    This class represents a recommendation system for planning a trip, including hotel, location, and restaurant recommendations.

    Attributes:
        hotels (Hotels): An instance of the Hotels class for hotel-related tasks.
        locations (Locations): An instance of the Locations class for location-related tasks.
        restaurants (Restaurant): An instance of the Restaurant class for restaurant-related tasks.
        trip (TripInformation): An instance of the TripInformation class containing trip details.
        restaurantDF (DataFrame): A DataFrame to store all restaurant data.
        daily_activities (list): A list to store daily trip activities.

    Methods:
        execute(): Execute a series of tasks to plan a trip, including hotel, location, and restaurant recommendations.
        choose_activities(type): Choose trip activities based on the distance matrix and restaurant types.
        execTripInformation(): Executes de TripInformation Class
        execHotels(): Executes the Hotel Class
        execLocation(): Executes the Location Class
        execRestaurants(): Executes the Restaurant Class
        rec(): Create a recommendation for each day of the trip, including restaurant choices.

    Static Methods:
        calculating_distances(): Calculate distances between all locations and restaurants and create a distance matrix.
  """

    def __init__(self):

        self.hotels = None
        self.locations = None
        self.restaurants = None
        self.trip = None

        self.restaurantDF = None
        self.restaurant_qtd = None
        self.daily_activies = []


    def execute(self):

        # Executes all the classes
        self.execTripInformation()

        self.execHotels()
        self.execLocation()
        self.execRestaurants()


        self.matrix = self.calculating_distances(self.locations.df, self.restaurants.normal.df)
        self.choose_activities(0)

        self.matrix = self.calculating_distances(self.locations.df, self.restaurants.fastfood.df)
        self.choose_activities(1)

        self.matrix = self.calculating_distances(self.locations.df, self.restaurants.michelin.df)
        self.choose_activities(2)

        self.rec()




    def execTripInformation(self):
        self.trip = TripInformation()
        self.trip.execute()

    def execHotels(self):
        self.hotels = Hotels(self.trip)
        self.hotels.execute()

    def execLocation(self):
        self.locations = Locations(self.trip)
        self.locations.execute()

    def execRestaurants(self):
        self.restaurants = Restaurant(self.trip)
        self.restaurants.execute()



    @staticmethod
    def calculating_distances(location_df, restaurant_df):

        distances, locations, restaurants = [], [], []

        # Loop that iterates over all Locations and Restaurants to calculate the distance(km) between them
        for index1, row1 in location_df.iterrows():
            for index2, row2 in restaurant_df.iterrows():
                locations.append(row1["name"])
                restaurants.append(row2["name"])
                distance = geopy.distance.geodesic((row1['latitude'], row1['longitude']), (row2['latitude'], row2['longitude'])).km + row1["distance"]
                distances.append(distance)

        # Creates a distance DataFrame
        distances_df = pd.DataFrame({'Location': locations,'Restaurant': restaurants, 'Distance (km)': distances})

        # Creates a matrix.
        matrix = distances_df.pivot_table(columns='Location', index='Restaurant', values="Distance (km)")

        return matrix

    def choose_activities(self, type):

        # function to choose the restaurant and the location each day
        # It will be used in the "rec" function to make a more complet recommendation

        # Gets the distance matrix
        stacked = self.matrix.stack()

        sorted_series = stacked.sort_values()
        restaurants = []
        places = []

        # Loop to get the location x restaurant combination with the shortest distance
        for index, value in sorted_series.iteritems():
            if len(restaurants) >= self.restaurants.normal.restaurant_qtd[type]:
                break
            row, column = index
            if not any(row in tup for tup in self.daily_activies):
                if not any(column in tup for tup in self.daily_activies):
                    restaurants.append(row)
                    places.append(column)
                    self.daily_activies.append((row, column))

    def rec(self):

        # Lists to store location and restaurant information
        locals, locals_lat, locals_lng = [], [], []
        restaurants, res_lat, res_lng, res_cuisine, rest_type = [], [], [], [], []

        counter = 0
        for j in range(3): # Loop over the 3 types of restaurants
            for i in range(self.restaurants.normal.restaurant_qtd[j]): # Loops over all the days each restaurant will have
                locals.append(self.daily_activies[counter][1]) # Puts the name of the restaurant and the location into the list
                restaurants.append(self.daily_activies[counter][0])
                if(j == 0): # If j == 0 (Normal Restaurant)
                    # Gets the info from the restaurant
                    restaurant_info = self.restaurants.normal.df[self.restaurants.normal.df["name"] ==
                                                      self.daily_activies[counter][0]]
                    restaurant_info["Cuisine"] = restaurant_info["Why"]
                    rest_type.append("Normal")
                if(j==1): # If j == 1 (FastFood)
                    restaurant_info = self.restaurants.fastfood.df[self.restaurants.fastfood.df["name"] ==
                                                        self.daily_activies[counter][0]]
                    restaurant_info["Cuisine"] = restaurant_info["type"]
                    rest_type.append("FastFood")
                if(j==2): # If j == 2 (Michelin)
                    restaurant_info = self.restaurants.michelin.df[self.restaurants.michelin.df["name"] ==
                                                        self.daily_activies[counter][0]]
                    restaurant_info["Cuisine"] = "Michelin awarded " + restaurant_info["cuisine"] + " Restaurant"
                    rest_type.append("Michelin")


                # Gets lat and lng from the location
                print(self.daily_activies[counter][1])
                location_info = self.locations.df[self.locations.df["name"] ==
                                                      self.daily_activies[counter][1]]
                locals_lat.append(location_info["latitude"].values[0])
                locals_lng.append(location_info["longitude"].values[0])

                # Gets the lat, lng and cuisine of the restaurant
                res_lat.append(restaurant_info["latitude"].values[0])
                res_lng.append(restaurant_info["longitude"].values[0])
                res_cuisine.append(restaurant_info["Cuisine"].values[0])
                counter +=1

        # Make a DataFrame with all the information
        days = pd.DataFrame({"Location Name": locals, "Restaurant": restaurants, "Restaurant cuisine": res_cuisine,
                             "Restaurant Type": rest_type,
                             "Restaurant lat": res_lat, "Restaurant lng": res_lng,
                             "Location lat": locals_lat, "Location lng": locals_lng
                             })

        # Put hotel information on the DataFrame
        days["Hotel"] = self.hotels.df["name"]
        days["Hotel lat"] = self.hotels.df["latitude"]
        days["Hotel lng"] = self.hotels.df["longitude"]

        # Shuffle the DataFrame to randomize each day
        days = days.sample(frac=1).reset_index(drop=True)

        self.dayDF = days



In [ ]:
class Plot:
    """
    This class handles plotting and visualization of trip recommendations.

    Attributes:
        recomendations (DataFrame): A DataFrame containing the trip recommendations.
        route_day (int): The day for which the route will be plotted.

    Methods:
        execute(): Execute the plotting and visualization tasks.
        print_recommendation(): Print the overall trip recommendation.
        plot_route(): Plot the route for a specific day of the trip.
    """


    def __init__(self, recomendations):
        self.recomendations =  recomendations.dayDF
        self.rec = recomendations
        self.route_day = 1


    def execute(self):
        # Print the overall trip recommendation
        self.print_recommendation()
        while True:
            change_day = int(input("Do you want to change the location of a certain day? (-1 do exit): "))
            if(change_day == -1):
                break

            self.change_location(change_day)
            self.print_recommendation()

        # Asks the user if they want to see the route of a specific day
        while True:
            plot_route = int(input("Do you want to see the route of a specific day? (-1 to exit): "))
            if(plot_route == -1):
                break

            self.route_day = plot_route-1
            self.plot_route()


    def print_recommendation(self):
        # Function to print the recommendations
        # Each Restaurant type has a different print
        print(f"For your {self.recomendations.shape[0]} day trip we recommend you to: \n")
        print(f"\tStay at the {self.recomendations['Hotel'][0]} Hotel, one of the best reviewed hotels of the region.")
        print("\n")
        print("And each day, we recommend you to do this activities: ")
        print("\n")
        for i in range(self.recomendations.shape[0]):
            print(f"Day {i+1}:")
            if(self.recomendations["Restaurant Type"][i] == "Normal"):
                print(f"\tGo to {self.recomendations['Location Name'][i]} and eat at the {self.recomendations['Restaurant'][i]}, \n\t"
                    f"one of the best reviwed {self.recomendations['Restaurant cuisine'][i]} restaurants of the region")
            if(self.recomendations["Restaurant Type"][i] == "FastFood"):
                print(f"\tGo to {self.recomendations['Location Name'][i]} and eat at the "
                      f"{self.recomendations['Restaurant'][i]} for a quick bite")
            if(self.recomendations["Restaurant Type"][i] == "Michelin"):
                print(f"\tGo to {self.recomendations['Location Name'][i]} and eat at the {self.recomendations['Restaurant'][i]}, "
                      f"{self.recomendations['Restaurant cuisine'][i]}")

        print("\n")

    def change_location(self, day):

        locations = self.recomendations["Location Name"].to_list()
        filtered_df = self.rec.locations.df[~rec.locations.df["name"].isin(locations)]

        new_location = filtered_df.iloc[random.randint(0,(filtered_df.shape[0]-1)), [0, 5,6 ]][:].values

        self.recomendations.iloc[day-1, 0] = new_location[0]

        self.recomendations.iloc[day-1, 6] = new_location[1]
        self.recomendations.iloc[day-1, 7] = new_location[2]




    def plot_route(self):

        # Plots the route using Google API

        # Gets the day that will be plotted
        rec = self.recomendations.loc[self.route_day]

        # gets the latitude and longitude of each activity of the day
        lat1, lng1 = rec["Hotel lat"], rec["Hotel lng"]
        lat2, lng2 = rec["Location lat"], rec["Location lng"]
        lat3, lng3 = rec["Restaurant lat"], rec["Restaurant lng"]

        location1, location2, location3 = (lat1, lng1), (lat2, lng2), (lat3, lng3)
        locations = [location1, location2, location3]
        now = datetime.now()


        gmaps.configure(api_key='API-KEY')
        fig = gmaps.figure()

        # Get the directions layers
        layer = gmaps.directions.Directions(location1, location3,waypoints=[location2],
                                            mode="car", avoid="ferries", departure_time=now, show_markers=False,
                                            stroke_color="red", stroke_opacity=1.0, stroke_weight=3.0)

        # Get the markers layer
        marker_layer = gmaps.marker_layer(locations, label=["Hotel", "Location", "Restaurant"],
                                          display_info_box=True, info_box_content=[rec["Hotel"],
                                            rec["Location Name"], rec["Restaurant"]])
        fig.add_layer(marker_layer)
        fig.add_layer(layer)

        # Plot
        display(fig)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
rec = Recomendation()

In [ ]:
rec.execute()

In [ ]:
plot = Plot(rec)
plot.execute()

For your 5 day trip we recommend you to: 

	Stay at the Holiday Inn Express New York City Times Square Hotel, one of the best reviewed hotels of the region.


And each day, we recommend you to do this activities: 


Day 1:
	Go to Statue of Liberty National Monument and eat at the O'Hara's Restaurant and Pub, 
	one of the best reviwed American restaurants of the region
Day 2:
	Go to Empire State Building and eat at the Dunkin' Donuts for a quick bite
Day 3:
	Go to The High Line and eat at the The River Cafe, 
	one of the best reviwed Vegetarian Friendly restaurants of the region
Day 4:
	Go to Flatiron Building and eat at the Piccola Cucina Osteria, 
	one of the best reviwed Italian restaurants of the region
Day 5:
	Go to Grand Central Terminal and eat at the Ai Fiori, Michelin awarded Italian Restaurant


Do you want to change the location of a certain day? (-1 do exit): 3
For your 5 day trip we recommend you to: 

	Stay at the Holiday Inn Express New York City Times Square Hotel, one o

Figure(layout=FigureLayout(height='420px'))

Do you want to see the route of a specific day? (-1 to exit): 2


Figure(layout=FigureLayout(height='420px'))

Do you want to see the route of a specific day? (-1 to exit): -1
